In [3]:
import pickle
import numpy as np


### 4. Flask : usando JSON

**with**: nos permite abrir el archivo sin necesidad de abrir y cerrar, evitando errores.

https://www.geeksforgeeks.org/with-statement-in-python/

Recordemos estos argumentos para el **open** :

    'math_model.pkl' > El archivo que contiene el modelo serializado.

    'rb' > Es la forma de lectura en binario.

Para deserializar un pickle, debemos mantener abierto el archivo que contiene la data en forma de lectura binaria.

In [74]:
# vamos a cargar uno de los modelos en memoria a partir de un archivo pickle
# el archivo se mantiene abierto solo en el ambito del with, despues del bloque se cierra automaticamente.
with open('math_model.pkl', 'rb') as f_math:
    modelo_matematicas = pickle.load(f_math)

print(type(modelo_matematicas))
modelo_matematicas

<class 'sklearn.linear_model._coordinate_descent.ElasticNetCV'>


ElasticNetCV(alphas=array([1.000e-03, 1.020e-01, 2.030e-01, 3.040e-01, 4.050e-01, 5.060e-01,
       6.070e-01, 7.080e-01, 8.090e-01, 9.100e-01, 1.011e+00, 1.112e+00,
       1.213e+00, 1.314e+00, 1.415e+00, 1.516e+00, 1.617e+00, 1.718e+00,
       1.819e+00, 1.920e+00, 2.021e+00, 2.122e+00, 2.223e+00, 2.324e+00,
       2.425e+00, 2.526e+00, 2.627e+00, 2.728e+00, 2.829e+00, 2.930e+00,
       3.031e+00, 3.132e+00, 3.2...
       8.485e+00, 8.586e+00, 8.687e+00, 8.788e+00, 8.889e+00, 8.990e+00,
       9.091e+00, 9.192e+00, 9.293e+00, 9.394e+00, 9.495e+00, 9.596e+00,
       9.697e+00, 9.798e+00, 9.899e+00, 1.000e+01]),
             copy_X=True, cv=3, eps=0.001, fit_intercept=True, l1_ratio=0.5,
             max_iter=1000, n_alphas=100, n_jobs=None, normalize=False,
             positive=False, precompute='auto', random_state=None,
             selection='cyclic', tol=0.0001, verbose=0)

Para poner el modelo a disposición, debemos tener en cuenta como vamos a definir la comunicación

En nuestro caso vamos a responder a un **POST requests** cuyo contenido va a ser un json como este :

**{ "model" : "math",**
 
 **"dummies": \["1", "0", "1", "0", "0", "0", "0", "1", "1", "0", "0", "0", "1", "0", "1", "0", "1"\] }**
 
 En donde cada posicion de un elemento en el array representa una dummy ordenada.

Para poder trabajar con este json debemos parsear el contenido del post requests (o sea el contenido que nos va a solicitar un potencial cliente).

Vamos a utilizar la funcionalidad dada por la clase _requests_ de flask para tratar la petición.

La clase _requests_ tiene un metodo *get_json* que permite parsear el contenido de un request especificado como json.

In [15]:
from flask import Flask, jsonify, request
app = Flask('Predictor de examenes')
#aclaramos la ruta del recurso y el tipo: (podriamos poner /modelos/listener)
@app.route('/',methods=['POST'])
#solo con esta linea, ya tenemos un recurso recibiendo requests en "127.0.0.1/".

#defino la funcion que manejarara el request
def predict():
     # obtengo los datos del request post.
    # notar que en este contexto request contiene la informacion 
    # que viene de la peticion externa (el metodo get_json lo transforma en un diccionario)
    data = request.get_json(force=True)
    # transformamos el dato del json (un array de string) en un array de enteros de numpy
    # para que lo entienda el modelo, notar la forma del array y la transformacion de los tipos de datos
    X_para_prediccion = np.array(data['dummies']).reshape(1, -1).astype('int') 
    # implementamos una logica en donde elegimos un modelo (podrian ser los otros modelos, lect/escritura)
    if data['model'] == 'math':
        # asignamos el array de numpy que nos devuelve una prediccion
        output = modelo_matematicas.predict(X_para_prediccion)
         # esta prediccion es un array de un solo elemento
        prediccion = output[0]
        # Le damos forma de un diccionario para poder hacer el traspaso a json trivialmente
        output = {'prediccion': prediccion}
        
        # en esta linea, transformamos el diccionario en json con jsonify (funcionalidad de flask)
        # y respondemos el request con un json mediante este return
        # este json es incorporado en el cuerpo de la respuesta
        return jsonify(output)

    
# listo eso es todo!   

### En este punto vamos a poner a escuchar al servidor en 127.0.0.1:5000

Entre las cosas que vamos a ver por la salida estandar, va a ser la interaccion de nuestro servidor, con las peticiones externas. 

cuando sea ejecutada la proxima celda, la notebook se va a encontrar en estado de ejecución respondiendo peticiones, en este estado se le puede realizar un requests

In [75]:
# el puerto 5000 es por default
app.run(host='127.0.0.1', port=5001)

 * Serving Flask app "Ejemplo Bokeh" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5001/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Apr/2020 19:37:09] "POST / HTTP/1.1" 405 -
127.0.0.1 - - [30/Apr/2020 19:37:28] "POST / HTTP/1.1" 405 -
127.0.0.1 - - [30/Apr/2020 19:37:29] "POST / HTTP/1.1" 405 -
127.0.0.1 - - [30/Apr/2020 19:38:18] "POST / HTTP/1.1" 405 -


### Y si quiero usar los otros dos modelos? 

In [76]:
app = Flask('Predictor de examenes')
#aclaramos la ruta del recurso y el tipo: (podriamos poner /modelos/listener)
@app.route('/sarasa/',methods=['POST'])
#solo conesta linea, ya tenemos un recurso recibiendo requests en "127.0.0.1/".
#defino la funcion que manejarara el request
def predict():
    def math(X_para_prediccion):      
        with open('math_model.pkl', 'rb') as f_math:
            modelo_matematicas = pickle.load(f_math)

        output = modelo_matematicas.predict(X_para_prediccion)

        # esta prediccion es un array de un solo elemento
        prediccion = output[0]

        # Le damos forma de un diccionario para poder hacer el traspaso a json trivialmente
        output = {'prediccion': prediccion}

        # en esta linea, transformamos el diccionario en json con jsonify (funcionalidad de flask)
        # y respondemos el request con un json mediante este return
        # este json es incorporado en el cuerpo de la respuesta
        return jsonify(output)

    def read(X_para_prediccion):

        with open('read_model.pkl', 'rb') as f_math:
            modelo_read = pickle.load(f_math)

        output = modelo_read.predict(X_para_prediccion)

        # esta prediccion es un array de un solo elemento
        prediccion = output[0]

        # Le damos forma de un diccionario para poder hacer el traspaso a json trivialmente
        output = {'prediccion': prediccion}

        # en esta linea, transformamos el diccionario en json con jsonify (funcionalidad de flask)
        # y respondemos el request con un json mediante este return
        # este json es incorporado en el cuerpo de la respuesta
        return jsonify(output)

    def write(X_para_prediccion):

        with open('write_model.pkl', 'rb') as f_math:
            modelo_write = pickle.load(f_math)

        output = modelo_write.predict(X_para_prediccion)

        # esta prediccion es un array de un solo elemento
        prediccion = output[0]

        # Le damos forma de un diccionario para poder hacer el traspaso a json trivialmente
        output = {'prediccion': prediccion}

        # en esta linea, transformamos el diccionario en json con jsonify (funcionalidad de flask)
        # y respondemos el request con un json mediante este return
        # este json es incorporado en el cuerpo de la respuesta
        return jsonify(output)
    
    mtype = {'math' : math, 'read' : read , 'write' : write }     
    # obtengo los datos del request post.
    # notar que en este contexto request contiene la informacion 
    # que viene de la peticion externa (el metodo get_json lo transforma en un diccionario)
    data = request.get_json(force=True)

    # transformamos el dato del json (un array de string) en un array de enteros de numpy
    # para que lo entienda el modelo, notar la forma del array y la transformacion de los tipos de datos
    X_para_prediccion = np.array(data['dummies']).reshape(1, -1).astype('int')
    
    # implementamos una logica en donde elegimos un modelo (podrian ser los otros modelos, lect/escritura)
    model=mtype[data['model']]
    pred=model(X_para_prediccion)
    
    return pred


In [78]:
app.run(host='0.0.0.0', port=5001)

 * Serving Flask app "Predictor de examenes" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5001/ (Press CTRL+C to quit)
181.239.216.57 - - [30/Apr/2020 19:50:46] "POST / HTTP/1.1" 200 -
181.28.5.218 - - [30/Apr/2020 19:50:55] "POST / HTTP/1.1" 200 -
181.229.206.84 - - [30/Apr/2020 19:51:00] "POST / HTTP/1.1" 200 -
181.28.84.59 - - [30/Apr/2020 19:51:08] "POST / HTTP/1.1" 200 -
181.229.206.84 - - [30/Apr/2020 19:51:17] "POST / HTTP/1.1" 200 -
186.136.51.36 - - [30/Apr/2020 19:51:25] "POST / HTTP/1.1" 200 -
190.245.241.24 - - [30/Apr/2020 19:51:40] "POST / HTTP/1.1" 200 -
181.229.206.84 - - [30/Apr/2020 19:51:52] "POST / HTTP/1.1" 200 -
181.28.5.218 - - [30/Apr/2020 19:52:12] "POST / HTTP/1.1" 200 -
190.245.241.24 - - [30/Apr/2020 19:52:13] "POST / HTTP/1.1" 200 -
190.245.241.24 - - [30/Apr/2020 19:52:15] "POST / HTTP/1.1" 200 -
181.46.160.39 - - [30/Apr/2020 19:52:18] "POST / HTTP/1.1" 200 -
181.46.160.39 - - [30/Apr/2020 19:52:22] "POST / HTTP/1.1" 200 -
181.46.160.39 - - [30/Apr/2020 19:52:25] "POST / HTTP/1.1" 200 -
181.46.160.39 - - [30/Apr/20

### server GET

In [79]:
app = Flask('Servidor Get')
@app.route('/',methods=['GET'])

def hola():
   
    # obtengo los datos del request post.
    # notar que en este contexto request contiene la informacion 
    # que viene de la peticion externa (el metodo get_json lo transforma en un diccionario)
    data=request.args.to_dict()
    print(data)
    return data

app.run(host='0.0.0.0',  port=5002 )

 * Serving Flask app "Servidor Get" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5002/ (Press CTRL+C to quit)
181.239.216.57 - - [30/Apr/2020 20:00:55] "GET /?a=&b=0 HTTP/1.1" 200 -


{'a': '', 'b': '0'}


181.239.216.57 - - [30/Apr/2020 20:01:03] "GET /?a=45&b=0 HTTP/1.1" 200 -


{'a': '45', 'b': '0'}


...

{'a': '2', 'b': '0'}


181.46.160.39 - - [30/Apr/2020 20:03:26] "GET /?a=23&b=44 HTTP/1.1" 200 -
181.46.160.39 - - [30/Apr/2020 20:03:26] "GET /favicon.ico HTTP/1.1" 404 -


{'a': '23', 'b': '44'}


181.28.5.218 - - [30/Apr/2020 20:04:10] "GET /?a=hola&b=3 HTTP/1.1" 200 -


{'a': 'hola', 'b': '3'}


### un poco mas vistoso

In [81]:
from bokeh.embed import components
from bokeh.plotting import figure
from bokeh.resources import INLINE
from flask import Flask, jsonify, request, render_template
import numpy as np
from bokeh.models import Range1d

app = Flask('Ejemplo Bokeh')

@app.route('/',methods=['GET'])

def bokeh():
    data=request.args.to_dict()
    a=int(data['a'])
    b=int(data['b'])
    n=int(data['n'])
    x=np.linspace(0,10,n)
    y=a*np.sin(b*x)
    fig = figure(plot_width=1280, plot_height=600)
    fig.line(x,y)
    left, right, bottom, top = 0, 10, -10, 10
    fig.x_range=Range1d(left, right)
    fig.y_range=Range1d(bottom, top)

    # magia html del bokeh
    js_resources = INLINE.render_js()
    css_resources = INLINE.render_css()
    script, div = components(fig)
    html = render_template(
        'index.html',
        plot_script=script,
        plot_div=div,
        js_resources=js_resources,
        css_resources=css_resources,
    )
    return html.encode('utf8')

app.run(host='0.0.0.0',  port=5002)

 * Serving Flask app "Ejemplo Bokeh" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5002/ (Press CTRL+C to quit)
[2020-04-30 20:21:15,687] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "C:\Users\fernando\AppData\Local\Continuum\miniconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\fernando\AppData\Local\Continuum\miniconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\fernando\AppData\Local\Continuum\miniconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\fernando\AppData\Local\Continuum\miniconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\fernando\AppData\Local\Continuum\miniconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\fernando\AppData\Local\Continuum\mi